# GAN(generative adversarial network)入門

このテキストは画像生成手法であるGANについて述べます。GANは「機械学習分野において、この10年間でもっとも面白いアイデア」とまで言われています。

このテキストを読み終える頃にはGANのエレガントさに驚いていると思います。

## Auto encoderとはどう違うのか？

画像生成手法といえば、教師なし学習によるオートエンコーダーがそれまでは主流でした。しかしオートエンコーダーは、教師なし学習なので、入力画像よりは高精度な画像を生成することができません。しかも、完全に内挿の範疇です。

要するに入力された画像と違うニュアンスの画像を生成することはできません。

## GANは教師なし学習なのか？

ではGANは教師ありなのか？教師なしなのか？どちらでしょう。正解はどちらでもありません。

GANは半教師あり学習と呼ばれるものになります。半教師ありというのは、名前の通り、半分は教師あり学習で半分は教師なし学習を使用したものです。

半分は教師あり、半分は教師なしということはDNN的には

NNの半分までは教師ありで、残り半分は教師なし
教師ありで学習されたNNと教師なしで学習されたNNの両方を用いる
みたいなことが考えられると思います。GANの場合は後者です。
要するに教師あり学習で学習されたNNと教師なし学習で学習されたNNの二つを使用していることになります。

## 二つのNNの役割

では2つのNNの役割について説明します。名前から入った方がわかりやすいと思います。

教師あり学習の方のNNはdiscriminator(判別器)　、教師なし学習の方のNNはgenerator(生成器)　と呼ばれています。

以下で、それぞれのNNについて説明しますが、説明後に図が示してありますので焦らず．

### generator

日本語で言うと生成器になります。何を生成するのかというと主に画像です。最初は画素値として完全に乱数で生成されます。以下のような画像です。

![](images/image1_1.jpg)

### dicriminator

日本語でいうと「判別器」です。名前の通り、判別します。何を判別するかというと、generatorから生成された画像が我々が持っている画像と同じものかどうかを判定します。

### アルゴリズム

上の二つのNNを使って、我々が知っているような画像を得るためのアルゴリズムを以下に示します。

- generatorが乱数で適当に画像を生成
- discriminatorにより、1で作成された画像が学習データと同じものかどうか判定する


**違うものとdiscriminatorが判定した場合**
- generatorはより高精度な（訓練画像に似ている）画像を生成できるように学習する
- 学習したgeneratorが画像を生成する
- discriminatorにより、4で作成された画像が学習データと同じものかどうか判定する
- 違うと判断されれば、また3に戻る。以後これを繰り返す。

**訓練データと同じものとdiscriminatorが判定した場合**
- discriminatorが見破ることができるようにdicriminatorが学習する。
- genertorが生成した画像を再度判定する。
- discriminatorが訓練データと同じものと判定した場合、再度見破ることができるようにdiscriminatorが学習する。なお無事に見破ることができた場合はgeneratorがより高精度な画像を生成するように学習する。

以上のアルゴリズムからわかるように、二つのNNが相互に影響を及ぼしあいながら学習が進んでいます。まるで二つのNNが敵対（adversarial）しているようです。これが名前の由来になります。

discriminatorが偽造紙幣を見破る警察、generatorが偽造紙幣を作る業者という例で説明されている記事もあります。

![](images/image1_2.png)

## どのように学習するか？

二つのNNが相互的に学習をしていくのはイメージできたかと思います。ここからは二つのNNがどのように学習していくのかを見ていきます。

GANの損失関数は以下のように定義されます。

$$ \mathrm{min}_{G}\mathrm{max}_{D}V(D,G) = E_{x{\sim}p_\mathrm{data}(x)}[\mathrm{log}D(x)]+ E_{z{\sim}p_\mathrm{z}(z)}[\mathrm{log}(1-D(G(z)))] $$

今はこういうものだと思った方が良いと思います（特に，なぜlogを使っているのか）。理論的になぜこの損失関数でうまくいくのか説明している論文がありましたが，初めてGANが生まれた時は少なからず「仮説だけど，損失をこれで置いたらうまくいった」という部分があったことは頭に入れておいてください。

- Dがうまく分類できると右辺第1項は、大きくなる。Dはdiscriminatorが学習データだと判断する確率。generatorという観点で見れば、バレることになるのでD(G(z))は小さくなります、要するに、log(1-D(G(z)))は大きくなるということです。
- Gが訓練データと似ている画像を生成できるようになると、Dは間違えるので、D(G(z))は大きくなる。すなわち、log(1-D(G(z)))は小さくなる

以上の1、2を繰り返し、DとGが相互に影響を及ぼしながら学習していきます。discriminator、generatorどちらか片方のlossを最大化もしくは最小化するのではなく、両方同時にというのがきもだと思います。だからこそ、右辺が2項に別れているわけです。

## 論文のコード

ここでは、GANの原著論文に書かれているアルゴリズムを詳細に追っていこうと思います。

[https://arxiv.org/pdf/1406.2661.pdf](https://arxiv.org/pdf/1406.2661.pdf)

![](images/image1_3.png)

### Discriminator(判別器)の更新式

- 事前確率分布$p_g(z)$ からミニバッチ分の $Z(z_1$~$z_m)$ を生成します  
→実際には1枚の画像のピクセル数×バッチ枚数分のノイズ

- 学習データXから$x_1$~$x_m$（1で取り出したバッチ枚数)だけ取り出します  
→この時の事前分布は1と同じである必要があります。要するに1でノイズを生成する時に正規分布に基づいてノイズをサンプリングしているなら、学習データをサンプリングする際も、同じ確率分布（今は正規分布）でサンプリングする必要があります。

- SGD（確率的勾配降下法）によりdiscriminatorを更新する  
各画像（1バッチ）に対して毎回更新ではなく、バッチ枚数ごとのlossを足してそれをmで割っているので、更新には、平均が用いられていると考えて良いでしょう。GANのデメリットに「学習時における、収束の不安定さ」が取り上げられますが、それがこの式からイメージできると思います。バッチ枚数が多ければ多いほど、各画像におけるLossの偏差は小さくなるので、学習を速くしたい場合、バッチサイズを大きくすれば良さそうですが、そうすると、1エポックにかかる時間が多くなるので、ビミョーなところだと言えます。

![](images/image1_4.png)

式1〜mのばらつきはmが大きくなるにつれて、小さくなりますが、バッチ内で学習を安定化させるためにmを大きくするとバッチが変わる度にバッチの特徴を表現した平均値が更新されるので、サンプリングされた画像が全く異なれば、次に生成される画像はうまく更新を反映できない（学習が進んだ後でも変な画像が時々生成されるのはこのため）可能性あります。

バッチを大きくすると、メモリの使用量が大きくなるデメリットがありますが、計算時間の効率化や比較的安定しやすいというメリットがあります。なのでGANにおける学習ではバッチサイズは大きめにしておくのがセオリーと言えます。

### generator(生成器)の更新式

- 事前分布$p_g(z)$からバッチサイズ分のノイズ（画素数×バッチサイズ）をサンプリング
- 以下の式によりgeneratorを更新する

![](images/image1_5.png)

上式の$D(G(z^{(i)}))$はノイズzがGに入力されgeneratorが画像を生成し、その画像をdiscriminatorに入力として与えられ、そしてそれが本物（トレインの画像）かどうかを判断する部分です。

generatorは最小化したいので（最初のLossの式参照）、$1-D(G(z^{(i)}))\simeq 0$となるように最適化していきます。要するに，似ている画像をgeneratorが生成するようになると、$D(G(z^{(i)}))$は1に近づく。

$1 - D(G(z^{(i)}))$を各画像に対して計算し、mごと（ミニバッチごと）の平均をそのバッチにおける更新量とします。

以上で述べた片方を最小化、片方は最大化というフローに乗っ取り、両者（両DNN）の均衡点を探すのがGANのオリジナリティと言えます。

次はフレームワークを使ってGANの実装を見ていきましょう。